## Introduction

Globally and even in national parks, ecosystems face major challenges due to climate change. The purpose of this study is to examine and illustrate how climate change will affect the national parks of southeastern Utah, which include Arches National Park, Canyonlands National Park, Hovenweep National Monument, and Natural Bridges National Park. We examine changes in soil moisture, vegetation cover, temperature and precipitation by looking at past data and future climate projections. Our goal is to provide information that enables park managers and conservationists to make informed decisions to reduce the impact of climate change on these important ecosystems.

## Data Overview

This project utilizes two key datasets to analyze the impact of climate change on Southeastern Utah National Parks. The first dataset, "NABR_historic.csv," contains historical climate data, providing a baseline for understanding past climate conditions and trends. The second dataset, "nearterm_data_2020-2024.csv," includes near-term climate projections, offering insights into expected future changes based on different Representative Concentration Pathway (RCP) scenarios. By combining these datasets, we can compare historical data with projected future conditions, allowing for a comprehensive analysis of climate trends and their potential effects on the park ecosystems. (@source)

In [56]:
import pandas as pd
df = pd.read_csv('Data/NABR_historic.csv')
df.head()


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Annual,T_Winter,T_Summer,T_Annual,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,6.37,1.630333,24.50402,24.50402,36.89,NaN,NaN,NaN,NaN,NaN
1,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-12.77,0.114652,0.078764,0.043514,0.051281
2,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,3.09,1.389056,24.11043,24.11043,37.95,NaN,NaN,NaN,NaN,NaN
3,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-18.96,0.130221,0.096412,0.041232,0.092241
4,-110.0472,37.60413,2021,NT,4.5,sc24,0,0,84,5,...,6.87,-0.334389,25.54266,10.31321,37.74,NaN,NaN,NaN,NaN,NaN


In [2]:
df2 = pd.read_csv('Data/nearterm_data_2020-2024.csv')
df2.head()

merged_df = pd.concat([df, df2], ignore_index=True)

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

merged_df.dropna(how='all', inplace=True)


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Annual,T_Winter,T_Summer,T_Annual,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,6.37,1.630333,24.50402,24.50402,36.89,NaN,NaN,NaN,NaN,NaN
1,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-12.77,0.114652,0.078764,0.043514,0.051281
2,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,3.09,1.389056,24.11043,24.11043,37.95,NaN,NaN,NaN,NaN,NaN
3,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-18.96,0.130221,0.096412,0.041232,0.092241
4,-110.0472,37.60413,2021,NT,4.5,sc24,0,0,84,5,...,6.87,-0.334389,25.54266,10.31321,37.74,NaN,NaN,NaN,NaN,NaN


## Data filtering process

To begin data cleaning, we downloaded two datasets: nearterm_data_2020-2024.csv, which contained near-term climate projections, and NABR_historic.csv, which contained historical climate data. These datasets were combined into a single DataFrame, removing all rows with missing values ​​and ensuring that there were no duplicate fields. This first step removed duplicates and completely empty records by merging relevant data.


Next we determined which columns are appropriate for each aggregation technique. For the columns representing vegetation (Canopy, Ann_Herb, Bare, Herb, Litter, Arbusb), we determined the highest values ​​for each group. We used a custom function to correctly remove nulls and NaNs from averages of other columns. The DataFrame was then grouped using the appropriate aggregation functions for long, lat, year, TimePeriod, RCP and scenario.

Finally, we cleaned the dataset by removing the DrySoilDays_Summer_whole, PPT_Annual, and T_Annual columns that had large missing values ​​or were not needed for further analysis. Possible missing values ​​ExtremeShortTermDryStress_Summer_whole and NonDrySWA_Summer_whole were filled with the mean of the corresponding column. This comprehensive method provided a clean, consistent and complete data set that allowed for an in-depth study of climate impacts on the Southeastern Utah Group of National Parks.

In [4]:
# List of columns to take the maximum value
max_columns = ['treecanopy', 'Ann_Herb', 'Bare', 'Herb', 'Litter', 'Shrub']

# List of columns to take the average, excluding max_columns and group by columns
group_by_columns = ['long', 'lat', 'year', 'TimePeriod', 'RCP', 'scenario']
avg_columns = [col for col in merged_df.columns if col not in max_columns + group_by_columns]

# Custom aggregation function to ignore zeros and NaNs for average
def custom_avg(series):
    return series[series != 0].mean()

# Group by the specified columns and apply aggregation
aggregated_df = merged_df.groupby(group_by_columns).agg(
    {**{col: 'max' for col in max_columns},
     **{col: custom_avg for col in avg_columns}}
).reset_index()

aggregated_df.drop(columns=['DrySoilDays_Summer_whole', 'PPT_Annual', 'T_Annual'], inplace=True)

# Fill missing values for specific columns with their column mean
aggregated_df['ExtremeShortTermDryStress_Summer_whole'].fillna(aggregated_df['ExtremeShortTermDryStress_Summer_whole'].mean(), inplace=True)
aggregated_df['NonDrySWA_Summer_whole'].fillna(aggregated_df['NonDrySWA_Summer_whole'].mean(), inplace=True)
aggregated_df


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Winter,PPT_Summer,T_Winter,T_Summer,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,1980,Hist,historical,sc1,0,0,84,5,...,13.79,2.69,0.964835,23.15924,37.05,-12.45,0.113447,0.096831,0.041876,0.052298
1,-110.0472,37.60413,1981,Hist,historical,sc1,0,0,84,5,...,2.25,9.39,3.334444,23.27065,37.55,-9.35,0.049382,0.060727,0.042639,0.093671
2,-110.0472,37.60413,1982,Hist,historical,sc1,0,0,84,5,...,4.12,9.50,-0.015556,22.05707,36.65,-16.55,0.109234,0.074817,0.045610,0.067789
3,-110.0472,37.60413,1983,Hist,historical,sc1,0,0,84,5,...,7.09,10.22,0.409444,21.32826,34.55,-15.05,0.120418,0.102542,0.044141,0.074802
4,-110.0472,37.60413,1984,Hist,historical,sc1,0,0,84,5,...,4.77,9.49,-1.047253,21.95978,35.35,-18.45,0.120209,0.077842,0.043179,0.070366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22482,-109.9659,37.62525,2024,NT,8.5,sc57,18,0,37,18,...,6.25,8.49,3.372912,23.75413,38.02,-12.16,0.142255,0.173792,0.091433,0.106907
22483,-109.9659,37.62525,2024,NT,8.5,sc58,18,0,37,18,...,7.49,4.13,2.162582,26.91342,46.77,-12.16,0.166254,0.158146,0.086355,0.095905
22484,-109.9659,37.62525,2024,NT,8.5,sc59,18,0,37,18,...,16.28,9.70,1.209121,23.48859,37.08,-10.38,0.151342,0.175814,0.091992,0.109333
22485,-109.9659,37.62525,2024,NT,8.5,sc60,18,0,37,18,...,2.00,8.15,1.549945,23.86120,37.06,-10.61,0.122249,0.104884,0.087115,0.118908


## Temperature Overview

In [22]:
import plotly.express as px
import plotly.graph_objects as go

filtered_df = aggregated_df
avg_temp = filtered_df.groupby(['year', 'RCP'])[['T_Summer', 'T_Winter']].mean().reset_index()

# Create interactive line chart for summer temperatures
fig_summer_temp = px.line(
    avg_temp,
    x='year',
    y='T_Summer',
    color='RCP',
    labels={'T_Summer': 'Summer Temperature (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Temperature Trends'
)
fig_summer_temp.update_layout(legend_title_text='RCP')

# Create interactive line chart for winter temperatures
fig_winter_temp = px.line(
    avg_temp,
    x='year',
    y='T_Winter',
    color='RCP',
    labels={'T_Winter': 'Winter Temperature (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Temperature Trends'
)
fig_winter_temp.update_layout(legend_title_text='RCP')
fig_winter_temp.write_html("fig_winter_temp.html")
fig_summer_temp.write_html("fig_summer_temp.html")



<iframe src="./fig_winter_temp.html" width="120%" height="600px"></iframe>
<iframe src="./fig_summer_temp.html" width="120%" height="600px"></iframe>

Differences between historical data and future estimates for the Representative Concentration Pathway (RCP) 8.5 and 4.5 scenarios are illustrated in interactive line graphs of summer and winter temperatures. In the absence of greater efforts to reduce emissions, RCP 8.5 predicts a trajectory with high greenhouse gas concentrations. The graphs show a steady increase in average temperatures over time, and this has clearly accelerated in recent years. The RCP 8.5 scenario shows a clearer trend, suggesting faster temperature increases if emissions continue. According to this study, the parks are likely to experience warmer summers and winters, which could significantly affect ecosystems.

## Seasonal Precipitation Analysis

In [25]:
filtered_df = aggregated_df
avg_precipitation = filtered_df.groupby(['year', 'RCP'])[['PPT_Summer', 'PPT_Winter']].mean().reset_index()

# Create interactive bar chart for summer precipitation
fig_summer_ppt = px.bar(
    avg_precipitation,
    x='year',
    y='PPT_Summer',
    color='RCP',
    barmode='group',
    labels={'PPT_Summer': 'Average Summer Precipitation (cm)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Precipitation'
)
fig_summer_ppt.update_layout(legend_title_text='RCP')

# Create interactive bar chart for winter precipitation
fig_winter_ppt = px.bar(
    avg_precipitation,
    x='year',
    y='PPT_Winter',
    color='RCP',
    barmode='group',
    labels={'PPT_Winter': 'Average Winter Precipitation (cm)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Precipitation'
)
fig_winter_ppt.update_layout(legend_title_text='RCP')

# Save the figures as HTML files
fig_summer_ppt.write_html("fig_summer_ppt.html")
fig_winter_ppt.write_html("fig_winter_ppt.html")

<iframe src="./fig_winter_ppt.html" width="120%" height="600px"></iframe>
<iframe src="./fig_summer_ppt.html" width="120%" height="600px"></iframe>

Interactive line charts of summer and winter temperatures revealed increasing trends in average temperature over time, especially when considering the RCP 8.5 scenario. These increases in temperature are expected to significantly affect the amount of precipitation needed to maintain the ecological balance in the national parks of Southeastern Utah. We examine these patterns by comparing historical and projected summer and winter precipitation using seasonal precipitation analysis. The bar graphs show projected changes in precipitation over time and highlight differences between historical data and projected estimates for the RCP 4.5 and 8.5 scenarios. These changes are important to understand because fluctuations in precipitation can affect overall ecosystem health, vegetation growth, and water availability. We can get a comprehensive overview of the potential effects of climate change on these protected areas by looking at changes in temperature and precipitation.

## Soil Moisture Dynamics

In [55]:
# Group by year and RCP, and calculate average volumetric water content
avg_soil_moisture = aggregated_df.groupby(['year', 'RCP'])[['VWC_Winter_whole', 'VWC_Spring_whole', 'VWC_Summer_whole', 'VWC_Fall_whole']].mean().reset_index()

# Create interactive line charts for soil moisture in different seasons
fig_soil_moisture_winter = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Winter_whole',
    color='RCP',
    labels={'VWC_Winter_whole': 'Winter Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Soil Moisture'
)
fig_soil_moisture_winter.update_layout(legend_title_text='RCP')

fig_soil_moisture_spring = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Spring_whole',
    color='RCP',
    labels={'VWC_Spring_whole': 'Spring Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Spring Soil Moisture'
)
fig_soil_moisture_spring.update_layout(legend_title_text='RCP')

fig_soil_moisture_summer = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Summer_whole',
    color='RCP',
    labels={'VWC_Summer_whole': 'Summer Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Soil Moisture'
)
fig_soil_moisture_summer.update_layout(legend_title_text='RCP')

fig_soil_moisture_fall = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Fall_whole',
    color='RCP',
    labels={'VWC_Fall_whole': 'Fall Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Fall Soil Moisture'
)
fig_soil_moisture_fall.update_layout(legend_title_text='RCP')

# Save the figures as HTML files
fig_soil_moisture_winter.write_html("fig_soil_moisture_winter.html")
fig_soil_moisture_spring.write_html("fig_soil_moisture_spring.html")
fig_soil_moisture_summer.write_html("fig_soil_moisture_summer.html")
fig_soil_moisture_fall.write_html("fig_soil_moisture_fall.html")



<iframe src="./fig_soil_moisture_winter.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_spring.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_summer.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_fall.html" width="120%" height="600px"></iframe>

After examining seasonal precipitation, we move on to soil moisture dynamics, which are essential to the well-being of park ecosystems. Soil moisture affects the sustainability of the entire ecosystem, plant growth and water availability. We can learn more about how these variations can affect plant and soil stability in the park by looking at volumetric water content (VWC) in winter, spring, summer and fall, along with historical and projected values.

Interactive line charts show that soil moistures in each season have been significant in recent decades. Future estimates under the RCP 8.5 scenario show a decrease in soil moisture, especially in summer. This pattern indicates that more severe drought could occur in the parks, stressing plants and possibly changing species composition. An extensive seasonal study highlights the importance of understanding and reducing the impact of climate change on soil moisture to protect the park's natural values.

## Extreme Weather Events analysis

In [54]:
# Group by year and RCP, and calculate average extreme weather events
avg_extreme_weather = aggregated_df.groupby(['year', 'RCP'])[['FrostDays_Winter', 'ExtremeShortTermDryStress_Summer_whole']].mean().reset_index()
# Create interactive bar chart for winter frost days
fig_frost_days = px.bar(
    avg_extreme_weather,
    x='year',
    y='FrostDays_Winter',
    color='RCP',
    barmode='group',
    labels={'FrostDays_Winter': 'Number of Frost Days', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Frost Days'
)
fig_frost_days.update_layout(legend_title_text='RCP')

# Create interactive bar chart for summer extreme short-term dry stress
fig_dry_stress = px.bar(
    avg_extreme_weather,
    x='year',
    y='ExtremeShortTermDryStress_Summer_whole',
    color='RCP',
    barmode='group',
    labels={'ExtremeShortTermDryStress_Summer_whole': 'Extreme Short-Term Dry Stress (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Extreme Short-Term Dry Stress'
)
fig_dry_stress.update_layout(legend_title_text='RCP')

# Save the figures as HTML files
fig_frost_days.write_html("fig_frost_days.html")
fig_dry_stress.write_html("fig_dry_stress.html")


<iframe src="./fig_frost_days.html" width="120%" height="600px"></iframe>
<iframe src="./fig_dry_stress.html" width="120%" height="600px"></iframe>

Our research ends with extreme weather events, especially cold days in winter and severe short-term drought stress in summer. These events can have negative effects on park ecosystems, affecting biodiversity, water availability and plant survival. We estimate the potential increase in these extreme events by comparing the projected future events of RCP 4.5 and RCP 8.5 with historical data.

The number of freezing days in winter has changed over time, as shown in the bar graphs, but is projected to decrease under the RCP 8.5 scenario. On the other hand, more extreme short-term drought stress is predicted in the summer, leading to warmer and drier weather. These results indicate that parks are likely to experience more frequent and severe droughts, which can increase plant mortality and change ecosystem structure. Understanding these patterns is important for developing tactics to strengthen park ecosystems against the harms of global warming.

## Climate Impact Features for 2024

In [53]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming 'aggregated_df' is the DataFrame that you have processed
# Filter the DataFrame for the year 2024
df_2024 = aggregated_df[aggregated_df['year'] == 2024]

# List of features to include in the dropdown menu
features = ['VWC_Winter_whole', 'VWC_Spring_whole', 'VWC_Summer_whole', 'VWC_Fall_whole', 
            'FrostDays_Winter', 'ExtremeShortTermDryStress_Summer_whole']

# Create the initial scatter map plot for the first feature in the list
initial_feature = features[0]

fig = px.scatter_mapbox(
    df_2024,
    lat="lat",
    lon="long",
    hover_name="year",
    hover_data={initial_feature: True, 'RCP': True},
    color=initial_feature,
    color_continuous_scale=[[0, 'white'], [1, 'red']],
    size_max=15,
    zoom = 11,
    mapbox_style="carto-positron"
)

# Create dropdown menu buttons for each feature
buttons = []
for feature in features:
    buttons.append(
        dict(
            method='update',
            label=feature,
            args=[
                {
                    'marker.color': [df_2024[feature]],
                    'hovertemplate': f'<b>2024</b><br>lat=%{{lat}}<br>long=%{{lon}}<br>RCP=%{{customdata[1]}}<br>{feature}=%{{marker.color}}<extra></extra>',
                }
            ]
        )
    )

# Update layout for better visibility and add the dropdown menu
fig.update_layout(
    title="Climate Impact Features for 2024",
    coloraxis_colorbar=dict(
        title="Feature Value",
    ),
    updatemenus=[dict(
        buttons=buttons,
        direction='down',
        showactive=True,
        x=0.4,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )]
)

# Save the figure as an HTML file
fig.write_html("climate_impact_features_map.html")






<iframe src="./climate_impact_features_map.html" width="120%" height="600px"></iframe>

Lastly, we have developed an interactive map for the year 2024 to understand the potential effects of climate change on the unique features of the National Parks of Southeastern Utah. This map allows users to select and view different climate effects in different areas of the park, such as extreme weather events and soil moisture. A drop-down menu allows users to select a function to track spatial variations and find the most affected locations. Because it shows areas that require more care and resources due to rapid climate change, this image is essential for future analysis and planning

## Conclusion

This project provides a comprehensive analysis of the potential impacts of climate change on the Southeastern Utah Group of National Parks, focusing on key factors such as temperature, precipitation, soil moisture and extreme weather events. Using historical data and future climate projections under different RCP scenarios, we identified important trends and changes that may affect the ecosystems of these parks.

The findings suggest a clear increase in temperature, especially under the RCP 8.5 scenario, which could lead to warmer summers and milder winters. This increase in temperature is likely to be accompanied by changes in precipitation, which can lead to a decrease in soil moisture, especially during the summer months. Such conditions can exacerbate drought stress, affect plant growth and change the composition of park vegetation. In addition, the projected increase in extreme weather events (eg, short-term drought stresses and reduced frost days) underscores the need for adaptive management strategies to mitigate their effects.

In general, this project emphasizes the importance of continuous monitoring and preventive measures to preserve the ecological integrity of the National Parks of Southeastern Utah. By understanding and addressing the potential impacts of climate change, park managers and conservationists can better protect these vital ecosystems for future generations. The interactive visualizations and analyzes presented in this study are valuable tools for informed decision-making and long-term planning in a changing climate.